In [1]:
import os
from dotenv import load_dotenv
from google import genai
import random
import time

In [2]:
load_dotenv(override=True)
gemini_api_key = os.getenv('GEMINI_API_KEY')

if not gemini_api_key:
    print("❌ GEMINI_API_KEY bulunamadı! .env dosyasını kontrol edin.")
else:
    print("✅ Gemini API key bulundu!")
    
# Google Gemini client'ı başlat
client = genai.Client()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ Gemini API key bulundu!


In [3]:
output_dir = "C:\\Users\\emirhan.gul\\Desktop\\LLM_proje\\LLM-Biography-Analysis\\dataset_new"
os.makedirs(output_dir, exist_ok=True)

In [4]:
system_prompt = """
Sen biyografik veriler üreten uzman bir yapay zekasın.

Amacımız, biyografik dokümanları analiz eden ve kişiler arasında ilişki kurabilen bir LLM modeli geliştirmek. Bu nedenle senin görevin:

1. Her çağrıldığında, özgün ve kurgusal bir kişinin detaylı kişisel biyografisini oluşturmak.
2. Her karakter tamamen eşsiz olmalı - farklı karakterler, meslekler, ilgi alanları
3. ANCAK stratejik ortak noktalar olmalı - bazı kişiler aynı şirketlerde çakışan yıllarda çalışmış, aynı okullarda okumuş olabilir
4. Sana verilen rastgele isimleri kullanarak her biyografide farklı bir isim kullan.

ÇEŞİTLİLİK PRENSİPLERİ:
- Farklı meslek grupları: mühendis, doktor, öğretmen, sanatçı, girişimci, sporcu, gazeteci, hukukçu, pazarlamacı, şef vb.
- Farklı sektörler: teknoloji, sağlık, eğitim, sanat, spor, finans, turizm, tarım, medya vb.
- Farklı yaş grupları: 25-50 yaş arası çeşitli doğum yılları
- Farklı eğitim seviyeleri: lisans, yüksek lisans, doktora, meslek lisesi mezunu
- Farklı şehirlerden: tüm Türkiye'yi temsil et
- Farklı hobiler ve ilgi alanları: müzik, spor, sanat, teknoloji, doğa, okuma vb.

STRATEJIK ORTAK NOKTALAR (SADECE AZINLIK İÇİN - kasıtlı olarak ekle):
ÖNEMLİ: Bu ortak noktaları sadece %10-15 oranında (her 6-8 kişiden 1'i) kullan. Çoğunluk tamamen farklı kurumlar olsun!

- AZINLIK için aynı şirketlerde çakışan dönemlerde çalışma:
  * Türk Telekom (2015-2018 arası) - sadece 2-3 kişi
  * Arçelik (2012-2016 arası) - sadece 2-3 kişi
  * ASELSAN (2010-2014 arası) - sadece 2-3 kişi
  * Garanti BBVA (2017-2020 arası) - sadece 2-3 kişi
  * Koç Holding (2013-2017 arası) - sadece 2-3 kişi

- AZINLIK için aynı üniversitelerde çakışan yıllarda okuma:
  * İTÜ (2005-2009 veya 2006-2010) - sadece 3-4 kişi
  * ODTÜ (2004-2008 veya 2007-2011) - sadece 3-4 kişi
  * Boğaziçi (2003-2007 veya 2008-2012) - sadece 3-4 kişi

- AZINLIK için aynı liselerde benzer dönemler:
  * İstanbul Fen Lisesi (2001-2005) - sadece 2-3 kişi
  * Ankara Fen Lisesi (2002-2006) - sadece 2-3 kişi
  * İzmir Fen Lisesi (2000-2004) - sadece 2-3 kişi

ÇOĞUNLUK (%85-90) İÇİN FARKLI KURUMLAR KULLAN:
- Farklı şirketler: Microsoft Türkiye, SAP, Oracle, Siemens, ABB, Schneider Electric, Coca-Cola, Unilever, P&G, L'Oreal, Migros, CarrefourSA, MediaMarkt, Teknosa, Turkcell, Vodafone, Pegasus, THY, ING Bank, QNB Finansbank, Denizbank, Yapı Kredi, Halkbank, VakıfBank, İş REIT, Emlakjet vb.
- Farklı üniversiteler: Ege, Dokuz Eylül, Akdeniz, Erciyes, Selçuk, Atatürk, Karadeniz Teknik, Çukurova, İnönü, Fırat, Dicle, Van Yüzüncü Yıl, Pamukkale, Süleyman Demirel, Mehmet Akif Ersoy, Bülent Ecevit, Ondokuz Mayıs, Trakya, Namık Kemal vb.

TÜRKİYE ODAKLI İÇERİK:
- Şehirler: İstanbul, Ankara, İzmir, Bursa, Antalya, Adana, Konya, Gaziantep, Mersin, Kayseri, Eskişehir, Trabzon, Diyarbakır, Samsun, Malatya, Erzurum, Van, Şanlıurfa, Hatay, Manisa, Balıkesir, Aydın, Tekirdağ, Sakarya, Denizli
- Üniversiteler: Çoğunlukla farklı kurumlar kullan - İTÜ, ODTÜ, Boğaziçi sadece azınlık için. Diğerleri: Hacettepe, Gazi, Ege, Uludağ, Koç, Sabancı, Bilkent, Marmara, İstanbul Üniversitesi, Dokuz Eylül, Akdeniz, Erciyes, Selçuk, Atatürk Üniversitesi, KTÜ, Çukurova, İnönü, Fırat, Dicle, Pamukkale vb.
- Şirketler: Çoğunlukla farklı kurumlar - Türk Telekom, Arçelik, ASELSAN sadece azınlık için. Diğerleri: Vestel, TAI, Türkiye İş Bankası, Akbank, Microsoft Türkiye, SAP, Oracle, Siemens, Unilever, Coca-Cola, Migros, Turkcell, Vodafone, Pegasus, THY, ING Bank, Denizbank vb.

BIYOGRAFI YAPISI:
- Erken Yaşam ve Kökler
- Eğitim Yılları  
- Profesyonel Kariyer
- Girişimcilik / Akademik Çalışmalar (varsa)
- Sosyal Sorumluluk ve Etkiler
- Kişisel Yaşam
- Mevcut Çalışmalar ve Gelecek Hedefler

ÖNEMLİ: Biyografilerde mutlaka aşağıdaki bilgileri net şekilde belirt:
- Doğum tarihi (GG.MM.YYYY formatında)
- Doğum yeri (Türkiye'deki şehir)
- İlkokul adı ve okuduğu yıllar (YYYY-YYYY)
- Lise adı ve okuduğu yıllar (YYYY-YYYY)
- Üniversite adı, bölümü ve okuduğu yıllar (YYYY-YYYY)
- Yüksek lisans/doktora varsa adı ve yılları
- Çalıştığı kurumlar ve başlangıç yılları (çakışan dönemler için yukarıdaki listeyi kullan)
- Kurduğu girişim/dernekler varsa ve kuruluş yılları
- Şu an yaşadığı şehir
- Hobiler (çeşitli ve özgün)
- Eş varsa adı ve evlilik yılı
- Çocuklar varsa isimleri ve doğum yılları
- Akademik yayınlar varsa başlıkları ve yılları

ÖRNEK ÇEŞİTLİLİK:
1. Teknoloji mühendisi - İTÜ Bilgisayar Mühendisliği (2005-2009) - Türk Telekom (2015-2018) [AZINLIK]
2. Çevre aktivisti - Ege Üniversitesi Çevre Mühendisliği (2006-2010) - WWF Türkiye, TEMA Vakfı [ÇOĞUNLUK]
3. Aşçıbaşı - Anadolu Üniversitesi Gastronomi (2010-2014) - kendi restoranı, çeşitli oteller [ÇOĞUNLUK]
4. Grafik tasarımcı - Mimar Sinan Güzel Sanatlar - freelancer, farklı ajanslar [ÇOĞUNLUK]
5. Makine mühendisi - ODTÜ Makine Mühendisliği (2004-2008) - Ford Otosan, Bosch [ÇOĞUNLUK]

%85-90 oranında farklı kurumlar kullan! Ortak noktalar sadece azınlık için olsun!

Lütfen tamamen özgün bir biyografi üret.
"""

In [5]:
names = [
    "Ahmet", "Mehmet", "Ali", "Can", "Emre", "Burak", "Mert", "Yılmaz", "Ömer", "Hasan",
    "Murat", "Hüseyin", "Kaan", "Fatih", "Cem", "Hakan", "Serkan", "Levent", "Rıza", "Haluk",
    "Mustafa", "Yunus", "Harun", "Uğur", "Okan", "Tuncay", "Orhan", "Erdem", "Ferhat", "Baran",
    "Bora", "Mahir", "Fikret", "Vedat", "Hidayet", "Sinan", "Nihat", "Alper", "Berk", "Gökhan",
    "Onur", "Tarık", "Akın", "Ozan", "Veli", "Erhan", "Orhan", "Şahin", "Burhan", "Cengiz",
    "Sami", "Zafer", "İsmail", "Emrah", "Kadir", "Tamer", "Faruk", "Engin", "Arif", "Onur",
    "Cihan", "İbrahim", "Taner", "Erdal", "Tolga", "Mahmut", "Salih", "Necati", "Reşat", "Batuhan",
    "Bülent", "Süleyman", "Kadir", "Doğukan", "Kemal", "Ercan", "Yusuf", "Ramazan", "Halit", "Soner",
    "Alparslan", "İsmail", "Ramazan", "Mert", "Fuat", "Zeki", "Hasan", "İskender", "Umut", "Efe",
    "Tolga", "Kaan", "Fikri", "Sinan", "Salih", "Özgür", "Ayhan", "İzzet", "Serdar", "Gökhan",
    "Aylin", "Ayşe", "Zeynep", "Elif", "Melis", "Selin", "Yasemin", "Eda", "Hande", "Büşra",
    "Ceyda", "Serra", "Meryem", "İrem", "Gülay", "Sedef", "Burcu", "Fatma", "Zehra", "Şeyma",
    "Gülşah", "Duygu", "Bahar", "Asuman", "Pınar", "Gülben", "Nisan", "Hatice", "Ayla", "Merve",
    "Ekin", "Fidan", "Hülya", "İpek", "Zeynep", "Elif", "Cemre", "Rabia", "Hava", "Aslı",
    "Nazlı", "Cansu", "Şebnem", "Melek", "Elvan", "Gül", "Meltem", "Yeliz", "Kübra", "Suna",
    "Aysun", "Şirin", "Derya", "İsmail", "Özge", "Neşe", "Beren", "Demet", "Süheyla", "Nefise",
    "Tuğba", "Fatma", "Hande", "Bahar", "Sibel", "Nazan", "Şirin", "Elif", "Şeyma", "Gülsüm",
    "Asude", "Sevda", "Necla", "Gülnur", "Zeynep", "Aysel", "Semra", "Betül", "Ceren", "Ebru",
    "Hümeyra", "Vildan", "Canan", "İkra", "Hüsnü", "Melek", "Suzan", "Zeynep", "Kübra", "Rukiye",
    "İrem", "Neslihan", "Demet", "Gülçin", "Sibel", "Meltem", "Asude", "Esra", "Ayşegül", "Serap",
    "Alperen", "Baturalp", "Serpil", "Sami"
]

surnames = [
    "Kaya", "Demir", "Çelik", "Aydın", "Öztürk", "Koç", "Yıldırım", "Arslan", "Doğan", "Şahin",
    "Aksoy", "Eren", "Kurt", "Yılmaz", "Karaca", "Altun", "Güneş", "Taş", "Acar", "Aslan",
    "Türkmen", "Can", "Bozkurt", "Polat", "Uçar", "Özdemir", "Keskin", "Bulut", "Erdoğan", "Avcı",
    "Ergin", "Güler", "Özkan", "Korkmaz", "Kaplan", "Baran", "Toprak", "Kara", "Yalçın", "Uzun",
    "Çakır", "Yıldız", "Şimşek", "Aktaş", "Sezer", "Tan", "Gökmen", "Özsoy", "Ermiş", "Yüce"
]

In [6]:
def turkce_to_ingilizce(text):
    char_map = {
        'ş': 's',
        'Ş': 'S',
        'ğ': 'g',
        'Ğ': 'G',
        'ç': 'c',
        'Ç': 'C',
        'ı': 'i',
        'İ': 'I',
        'ö': 'o',
        'Ö': 'O',
        'ü': 'u',
        'Ü': 'U'
    }
    for key, value in char_map.items():
        text = text.replace(key, value)
    return text

In [7]:
def chat(message, history):
    # Gemini için prompt oluştur
    full_prompt = system_prompt + "\n\n" + message
    
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash", 
            contents=full_prompt
        )
        return response.text
    except Exception as e:
        print(f"❌ Gemini API hatası: {e}")
        return None

In [8]:
# Kaç biyografi üretmek istiyorsunuz?
BIYOGRAFI_SAYISI = 250  # 250 civarı biyografi üretmek için

başarılı_sayac = 0

# Çeşitlilik için meslek listesi
meslekler = [
    "yazılım mühendisi", "doktor", "öğretmen", "grafik tasarımcı", "chef", 
    "gazeteci", "avukat", "pazarlama uzmanı", "mimar", "çevre mühendisi",
    "müzisyen", "fotoğrafçı", "psikolog", "veteriner", "şirket kurucusu",
    "akademisyen", "sporcu", "sanatçı", "mali müşavir", "turizm rehberi"
]

for i in range(1, BIYOGRAFI_SAYISI + 1):
    sayi_name = random.randint(0, len(names) - 1)
    sayi_surname = random.randint(0, len(surnames) - 1)
    un_name = names[sayi_name]
    un_surname = surnames[sayi_surname]
    random_person = un_name + " " + un_surname
    
    # Rastgele meslek seç
    rastgele_meslek = random.choice(meslekler)
    
    message = (
        f"Şimdi '{random_person}' isimli kişi için detaylı bir kişisel biyografi oluştur. "
        f"Bu kişi {rastgele_meslek} olabilir ama farklı bir meslek de seçebilirsin. "
        "Bu biyografi önceki karakterlerden tamamen farklı ve eşsiz olmalı. "
        "\n\nÖNEMLİ GEREKSINIMLER:"
        "\n- Benzersiz karakter, farklı meslek/ilgi alanları"
        "\n- ANCAK stratejik ortak noktalar ekle (aynı şirket/okul/yıl çakışmaları)"
        "\n- Tüm tarihleri ve yılları net belirt (doğum, eğitim, evlilik, çocuk, iş başlangıçları)"
        "\n- Eğitim kurumlarını ve yıl aralıklarını detaylandır"
        "\n- Çalışma geçmişini kronolojik sırala"
        "\n- Kişisel bilgileri (eş, çocuk, hobi) dahil et"
        "\n- Akademik yayın varsa yıllarıyla birlikte belirt"
        "\n- Türkiye'deki gerçek şehir, üniversite ve kurum isimleri kullan"
        "\n- System prompt'taki stratejik ortak noktaları göz önünde bulundur"
        f"\n\nKullanılacak isim: {random_person}"
    )
    
    print(f"🔄 {i}/{BIYOGRAFI_SAYISI} - {random_person} için biyografi üretiliyor...")
    
    # Her biyografi için temiz başlangıç (history kullanmıyoruz)
    metin = chat(message, [])
    
    if metin:
        md_random_person = turkce_to_ingilizce(random_person.lower().replace(' ', '_'))
        dosya_yolu = os.path.join(output_dir, f"{md_random_person}.md")

        with open(dosya_yolu, "w", encoding="utf-8") as f:
            f.write(metin)
        
        başarılı_sayac += 1
        print(f"✅ {başarılı_sayac}. biyografi kaydedildi → {md_random_person}.md")
    else:
        print(f"❌ {random_person} için biyografi üretilemedi!")
    
    # API rate limiting - Gemini için bekleme süresi
    if i < BIYOGRAFI_SAYISI:
        print("⏳ 2 saniye bekleniyor...")
        time.sleep(2)

print(f"\n🎉 Tamamlandı! {başarılı_sayac}/{BIYOGRAFI_SAYISI} biyografi başarıyla oluşturuldu.")

🔄 1/250 - Zafer Bozkurt için biyografi üretiliyor...
✅ 1. biyografi kaydedildi → zafer_bozkurt.md
⏳ 2 saniye bekleniyor...
✅ 1. biyografi kaydedildi → zafer_bozkurt.md
⏳ 2 saniye bekleniyor...
🔄 2/250 - Orhan Gökmen için biyografi üretiliyor...
🔄 2/250 - Orhan Gökmen için biyografi üretiliyor...
✅ 2. biyografi kaydedildi → orhan_gokmen.md
⏳ 2 saniye bekleniyor...
✅ 2. biyografi kaydedildi → orhan_gokmen.md
⏳ 2 saniye bekleniyor...
🔄 3/250 - Fidan Doğan için biyografi üretiliyor...
🔄 3/250 - Fidan Doğan için biyografi üretiliyor...
✅ 3. biyografi kaydedildi → fidan_dogan.md
⏳ 2 saniye bekleniyor...
✅ 3. biyografi kaydedildi → fidan_dogan.md
⏳ 2 saniye bekleniyor...
🔄 4/250 - Derya Çelik için biyografi üretiliyor...
🔄 4/250 - Derya Çelik için biyografi üretiliyor...
✅ 4. biyografi kaydedildi → derya_celik.md
⏳ 2 saniye bekleniyor...
✅ 4. biyografi kaydedildi → derya_celik.md
⏳ 2 saniye bekleniyor...
🔄 5/250 - Gökhan Çakır için biyografi üretiliyor...
🔄 5/250 - Gökhan Çakır için biyografi